In [1]:
# device_num = "0, 1"

# # !export CUDA_VISIBLE_DEVICES=device_num

# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
# os.environ["CUDA_VISIBLE_DEVICES"]=f"{device_num}"
# os.environ["CUDA_LAUNCH_BLOCKING"]="1"
import torch
print(torch.cuda.device_count())

#PYTHONUTF8=1
# os.environ['PYTHONUTF8'] = '1'

4


In [2]:
# device = "auto"
device = "cuda"

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, DataCollatorForLanguageModeling, BitsAndBytesConfig
from datasets import load_dataset, Dataset
# from peft import get_peft_model, LoraConfig, TaskType, PeftModel
# from unsloth import FastLanguageModel
import pprint
import json
from pathlib import Path
import transformers
import os
import re
from tqdm import tqdm
import time

# bits and bytes
import bitsandbytes

In [4]:
max_seq_length = 3000     # Unsloth auto supports RoPE Scaling internally!
# dtype = None              # None for auto detection
load_in_4bit = False      # Use 4bit quantization to reduce memory usage. Can be False.


# End

if torch.cuda.get_device_capability()[0] >= 8:
    attn_implementation = "flash_attention_2"
    torch_dtype = torch.bfloat16
else:
    attn_implementation = "eager"
    torch_dtype = torch.float16
# attn_implementation = "eager"
# attn_implementation = "flash_attention_2"
# torch_dtype = torch.bfloat16
print(f"Using {attn_implementation} for attention computation.")

BASE_DIR = "../finetuning/try_lora"

Using flash_attention_2 for attention computation.


In [5]:
common_prompt = """
나는 훌룡한 HVAC 관련 질문 답변을 제공하는 인공지능이다. 사용자의 질문을 받아들여서, 그에 맞는 답변을 제공하는 것이 내 임무이다. 사용자의 질문을 받아들일 때, 다음과 같은 절차를 따라야 한다.
<출력 내용>
1. 'Formalized Input': 사용자의 추상적 질문을 구체화 및 정규화한 결과. 다양한 형태의 질문들을 가장 핵심적이고 근본적인 질문으로 변환한 결과.
2. 'Input Semantic Parsing': Input Semantic Parsing 결과. dict 형태로 구성되며, Temporal, Spatial, Modality, Operation을 가짐.
3. 'Strategy': 질문을 해결하기 위한 전략을 답변 전 고민. Objective: 질문의 근본적 의도 및 답변의 목적. Expected Output: 답변의 예상 결과. Step: 답변을 위한 구체으로 쪼개진 단계.
4. 'Instruction Set': 문제 해결을 위해 나의 실제 행동.

<DDL statement>
CREATE TABLE IF NOT EXISTS data_t
(
    id integer NOT NULL DEFAULT nextval('data_t_id_seq'::regclass),
    idu_id integer,
    roomtemp double precision,
    settemp double precision,
    oper boolean,
    "timestamp" timestamp without time zone NOT NULL
)
    
CREATE TABLE IF NOT EXISTS idu_t
(
    id integer NOT NULL DEFAULT nextval('idu_t_id_seq'::regclass),
    name character varying(50) COLLATE pg_catalog."default",
    metadata character varying(255) COLLATE pg_catalog."default",
    CONSTRAINT idu_t_pkey PRIMARY KEY (id)
)

"""

current_metadata = json.load(open(f"{BASE_DIR}/metadata.json", "r"))

print(current_metadata)

{'site_name': 'YongDongIllHighSchool', 'user_name': '홍길동', 'user_role': 'customer', 'idu_name': '01_IB5', 'idu_mapping': {'01_IB5': ['우리반'], '01_IB7': ['옆반'], '02_IB1': ['앞반']}, 'modality_mapping': {'roomtemp': ['실내온도'], 'settemp': ['설정온도'], 'oper': ['전원']}, '말투': '존댓말', '언어': '한국어', 'current_datetime': '2022-09-30 12:00:00'}


In [6]:
responses = json.load(open(f"response.json", "r"))
for i, response in enumerate(responses):
    print(response)
# dataset_tr = json.load(open(f"{BASE_DIR}/training_dataset_v2_directsql_tr.json", "r"))
# for i, data in enumerate(dataset_tr):
#     dataset_tr[i]["Response"] = str(dataset_tr[i]["Response"])


{'Input': '지금 옆반 에어컨 상태 알려줘', 'Reference': {'Formalized Input': '지금 옆반의 에어컨 전원 상태 알려줘.', 'Input Semantic Parsing': {'Temporal': {'지금': '2022-09-30 12:00:00'}, 'Spatial': {'옆반': '01_IB7'}, 'Modality': {'에어컨 전원': 'oper'}}, 'Strategy': {'Objective': '현재 옆반의 에어컨 전원 상태를 조회함.', 'Expected Outputs': ['옆반의 에어컨은 현재 켜져있습니다.'], 'Step': ['옆반의 현재 에어컨 전원 상태 데이터를 쿼리해 qr_aircon_oper에 저장', 'qr_aircon_oper에서 에어컨 전원 상태 반환']}, 'Instruction Set': [{'type': 'q', 'run': {'args': {'table_name': 'data_t', 'columns': ['oper'], 'conditions': ["timestamp = '2022-09-30 12:00:00'"], 'subquery': "idu_id = (SELECT id FROM idu_t WHERE name = '01_IB7')"}}, 'result': {'name': 'qr_aircon_oper'}}, {'type': 'o', 'python_script_to_process': "oper_status = qr_aircon_oper['oper'].values[0]", 'return': ['oper_status']}, {'type': 'r', 'templates': ['옆반의 에어컨은 현재 {oper_status}입니다.']}]}, 'Candidate': {'Formalized Input': '지금 옆반의 에어컨 전원 상태 알려줘.', 'Input Semantic Parsing': {'Temporal': {'지금': '2022-09-30 12:00:00'}, 'Spatial': {'옆반':

# LLM as Judge accuracy

In [19]:
judge_model_ids = [
    # 'MLP-KTLim/llama-3-Korean-Bllossom-8B',
    # "Saxo/Linkbricks-Horizon-AI-Korean-Gemma-2-sft-dpo-27B",
    # "nbeerbower/gemma2-gutenberg-27B",
    # "google/gemma-2-27b",
    # "Saxo/Linkbricks-Horizon-AI-Llama-3.3-Korean-70B-sft-dpo",
    "deepseek-ai/DeepSeek-R1"
]




def model_dir(model_id):
    return f"/model/{model_id.replace('/', '-')}"

def get_judge_model_and_tokenizer(model_id):
    # pretrained_model, tokenizer = FastLanguageModel.from_pretrained(
    #     model_name = model_id,  
    #     max_seq_length = max_seq_length,
    #     dtype = dtype,
    #     load_in_4bit = load_in_4bit,
    #     # device_map=device,
    #     cache_dir=model_dir(model_id) + "/cache",
    #     attn_implementation=attn_implementation,
    #     # local_files_only=True
    # )
    # load_in_4bit = True
    
    
    
    if "llama" in model_id.lower():
        model = AutoModelForCausalLM.from_pretrained(
            model_id,
            torch_dtype=torch_dtype,  # dtype 적용
            cache_dir=model_dir(model_id) + "/cache",
            attn_implementation=attn_implementation,
            # load_in_4bit=load_in_4bit if 'bitsandbytes' in globals() else False,
            device_map=device,
        )
    elif "gemma" in model_id.lower():
        model = AutoModelForCausalLM.from_pretrained(
            model_id,  
            torch_dtype=torch_dtype,  # dtype 적용
            # load_in_4bit=True,
            # bnb_4bit_compute_dtype=torch.float16,
            quantization_config=BitsAndBytesConfig(
                # load_in_4bit=True,
                # bnb_4bit_use_double_quant=True,
                # bnb_4bit_quant_type="nf4",
                # bnb_4bit_compute_dtype=torch_dtype
                load_in_8bit=True,
                llm_int8_enable_fp32_cpu_offload=True
            ),
            cache_dir=model_dir(model_id) + "/cache",
            attn_implementation=attn_implementation,
            device_map=device,
        )
    else:
        raise ValueError(f"Unknown model_id: {model_id}")

    tokenizer = AutoTokenizer.from_pretrained(
        model_id,
        cache_dir=model_dir(model_id) + "/cache"
    )
    
    # for inference
    # FastLanguageModel.for_inference(pretrained_model)
    
    return model, tokenizer

judge_models_and_tokenizers = {}
for model_id in judge_model_ids:
    judge_models_and_tokenizers[model_id] = get_judge_model_and_tokenizer(model_id)

Fetching 25 files:   0%|          | 0/25 [00:00<?, ?it/s]

README.md:   0%|          | 0.00/2.90k [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

model-00001-of-00012.safetensors:   0%|          | 0.00/4.74G [00:00<?, ?B/s]

...

ggml-model-q4_k_m.gguf:   0%|          | 0.00/16.6G [00:00<?, ?B/s]

model-00003-of-00012.safetensors:   0%|          | 0.00/4.87G [00:00<?, ?B/s]

ggml-model-q8_0.gguf:   0%|          | 0.00/28.9G [00:00<?, ?B/s]

ggml-model-q5_k_m.gguf:   0%|          | 0.00/19.4G [00:00<?, ?B/s]

ggml-model-q6_k.gguf:   0%|          | 0.00/22.3G [00:00<?, ?B/s]

In [16]:

# Input: {responses[0]["Input"]}
# Metadata: {current_metadata}
# Reference: {responses[0]["Reference"]}

prompt_for_judge = f"""
너는 HVAC 관련 LLM의 성능을 평가하는 인공지능이다. 너는 특정 Input에 대해 Metadata, Reference 답변과 Candidate 답변을 비교해 다음 기준으로 평가한다.
아래는 예시일 뿐이며 답변시 input과 reference, candidate를 사용해 실제로 평가해야 한다. 예시를 그대로 따라하지 말것.

너는 다음과 같은 항목으로 평가한다.
Structural Coherence (0 or 1): Reference와 Candidate의 json 구조적(브라켓, 콜론, 쉼표) 일치도.  등의 일치도.
Structural Correctness (0 or 1): Candidate의 json 구조적 일치도. 실행 가능한지 여부.
Input Semantic Parsing Coherence (0 or 1): Reference 기준 Candidate의 Input Semantic Parsing 의 의미적 일치도.
Strategy Coherence (0 ~ 1): Reference 기준 Candidate의 전략의 의미적 일치도.
Instruction Set type q Correctness (0 ~ 1): Candidate 의 Instruction Set중 type q의 실행 가능 여부. 쿼리가 오류 없이 테이블에 맞게 실행되는지 여부.
Instruction Set type q Coherence (0 ~ 1): Reference 기준 Candidate의 Instruction Set중 type q의 의미적 일치도. 실행 가능한 쿼리중 결과가 의미적으로 일치하는지 여부.
Instruction Set type o Correctness (0 ~ 1): Reference 기준 Candidate의 Instruction Set중 type o의 실행 가능 여부. 파이썬 스크립트가 오류 없이 실행되는지 여부.
Instruction Set type o Coherence (0 ~ 1): Reference 기준 Candidate의 Instruction Set중 type o의 의미적 일치도. 실행되는 파이썬 스크립트의 결과가 의미적으로 일치하는지 여부.
Instruction Set type r Coherence (0 ~ 1): Reference 기준 Candidate의 Instruction Set중 type r의 의미적 일치도. 뱉어내는 결과의 의미적 일치도.

너는 다음과 같은 형식으로 이유와 평가값을 출력한다.
{{

    "Input": "현재 온도 알려줘"
    "Structural Coherence": ['두 입력은 구조적으로 일치한다.', 1], 
    "Structural Correctness": ['Candidate의 json 구조가 올바르다.', 1],
    "Input Semantic Parsing Coherence": ['필요한 모든 정보를 올바르게 추출했다.', 1],
    "Strategy Coherence": ['Objective과 Expected outputs는 올바르게 추출했지만, step의 일부가 누락되었다. 데이터를 계산하지 않고 반환한다.', 0.8], 
    "Instruction Set type q Correctness": ['쿼리 3개중 2개가 실행 가능하다. SELECT * from ABC 쿼리에 맞지 않는 테이블이 있다.', 0.67],
    "Instruction Set type q Coherence": ['실행가능한 쿼리 2개중 2개가 의미적으로 일치한다. Candidate의 SELECT IDU from data_t는 Reference의 SELECT IDU_ID from data_t 와 다르다.', 1.0],
    "Instruction Set type o Correctness": ['파이썬 스크립트 1개중 1개가 실행 가능하다. np.addtwo()는 없는 함수다.', 1.0],
    "Instruction Set type o Coherence": ['실행가능한 파이썬 스크립트 1개중 1개가 의미적으로 일치한다. pd.groupby()는 pd.sum()과 의미적으로 다르다.', 1.0],
    "Instruction Set type r Coherence": ['응답 1개중 1개가 의미적으로 일치한다.', 1.0],
}}

너는 원본 모델의 다음 프롬프트를 참고하여 성능을 평가한다.

나는 훌룡한 HVAC 관련 질문 답변을 제공하는 인공지능이다. 사용자의 질문을 받아들여서, 그에 맞는 답변을 제공하는 것이 내 임무이다. 사용자의 질문을 받아들일 때, 다음과 같은 절차를 따라야 한다.
<출력 내용>
1. 'Formalized Input': 사용자의 추상적 질문을 구체화 및 정규화한 결과. 다양한 형태의 질문들을 가장 핵심적이고 근본적인 질문으로 변환한 결과.
2. 'Input Semantic Parsing': Input Semantic Parsing 결과. dict 형태로 구성되며, Temporal, Spatial, Modality, Operation을 가짐.
3. 'Strategy': 질문을 해결하기 위한 전략을 답변 전 고민. Objective: 질문의 근본적 의도 및 답변의 목적. Expected Output: 답변의 예상 결과. Step: 답변을 위한 구체으로 쪼개진 단계.
4. 'Instruction Set': 문제 해결을 위해 나의 실제 행동.

<DDL statement>
CREATE TABLE IF NOT EXISTS data_t
(
    id integer NOT NULL DEFAULT nextval('data_t_id_seq'::regclass),
    idu_id integer,
    roomtemp double precision,
    settemp double precision,
    oper boolean,
    "timestamp" timestamp without time zone NOT NULL
)
    
CREATE TABLE IF NOT EXISTS idu_t
(
    id integer NOT NULL DEFAULT nextval('idu_t_id_seq'::regclass),
    name character varying(50) COLLATE pg_catalog."default",
    metadata character varying(255) COLLATE pg_catalog."default",
    CONSTRAINT idu_t_pkey PRIMARY KEY (id)
)

자 앞으로 input, candidate과 reference가 있는 글을 넘겨줄거야. 각 input에 대해서 위 포맷으로 답해줘. 시 작"""

print(prompt_for_judge)


너는 HVAC 관련 LLM의 성능을 평가하는 인공지능이다. 너는 특정 Input에 대해 Metadata, Reference 답변과 Candidate 답변을 비교해 다음 기준으로 평가한다.
아래는 예시일 뿐이며 답변시 input과 reference, candidate를 사용해 실제로 평가해야 한다. 예시를 그대로 따라하지 말것.

너는 다음과 같은 항목으로 평가한다.
Structural Coherence (0 or 1): Reference와 Candidate의 json 구조적(브라켓, 콜론, 쉼표) 일치도.  등의 일치도.
Structural Correctness (0 or 1): Candidate의 json 구조적 일치도. 실행 가능한지 여부.
Input Semantic Parsing Coherence (0 or 1): Reference 기준 Candidate의 Input Semantic Parsing 의 의미적 일치도.
Strategy Coherence (0 ~ 1): Reference 기준 Candidate의 전략의 의미적 일치도.
Instruction Set type q Correctness (0 ~ 1): Candidate 의 Instruction Set중 type q의 실행 가능 여부. 쿼리가 오류 없이 테이블에 맞게 실행되는지 여부.
Instruction Set type q Coherence (0 ~ 1): Reference 기준 Candidate의 Instruction Set중 type q의 의미적 일치도. 실행 가능한 쿼리중 결과가 의미적으로 일치하는지 여부.
Instruction Set type o Correctness (0 ~ 1): Reference 기준 Candidate의 Instruction Set중 type o의 실행 가능 여부. 파이썬 스크립트가 오류 없이 실행되는지 여부.
Instruction Set type o Coherence (0 ~ 1): Reference 기준 Candidate의 Instruction Set중 type o

In [17]:
def judge(model_id, model, tokenizer, input_text, ref, can):
    start_time = time.time()
    
    if "llama" in model_id.lower():
        chat = [
            {"role": "system", "content": f"{prompt_for_judge}"},
            {"role": "user", "content": f"metadata:{current_metadata};input:{input_text};ref:{ref};can:{can}"},
        ]
        
        # print(chat)
        
        inputs = tokenizer.apply_chat_template(
            chat,
            tokenize = True,
            add_generation_prompt = True, # Must add for generation
            return_tensors = "pt",
        ).to(model.device)
        
        # print(inputs)
        
        # outputs = model.generate(
        #     input_ids = inputs,
        #     max_new_tokens = 32,
        #     use_cache = True,
        #     # pad_token_id = tokenizer.eos_token_id,
        #     # eos_token_id = tokenizer.eos_token_id,
        # )
        
        # response = tokenizer.batch_decode(outputs)[0]
        # print(response)
        # # print("Extracting..")
        # parsed_response = extract_content(response)
        # pprint.pprint(f"Query: {query}, Time: {time.time() - start_time}")
        # print(parsed_response)
        
        # parsed_response_dict = eval(parsed_response)
        
        # return parsed_response, parsed_response_dict
        
        text_streamer = transformers.TextStreamer(
            tokenizer, 
            skip_prompt = True
        )
        _ = model.generate(
            input_ids = inputs, 
            streamer = text_streamer, 
            max_new_tokens = 300, 
            use_cache = True,
            pad_token_id = tokenizer.eos_token_id,
            eos_token_id = tokenizer.eos_token_id,
        )
        
    
    elif "gemma" in model_id.lower():
        # input_text = f"{prompt_for_judge};metadata:{current_metadata};input:{input_text};ref:{ref};can:{can}"
        chat = [
            # {"role": "system", "content": f"{prompt_for_judge}"},
            {"role": "user", 
             "content": f"{prompt_for_judge};metadata:{current_metadata};input:{input_text};ref:{ref};can:{can};",
            # "content": "Say Hi!"
            },
        ]
        
        inputs = tokenizer.apply_chat_template(
            chat,
            tokenize = True,
            add_generation_prompt = True, # Must add for generation
            return_tensors = "pt",
        ).to(model.device)
        
        
        text_streamer = transformers.TextStreamer(
            tokenizer, 
            skip_prompt = True
        )
        _ = model.generate(
            input_ids = inputs, 
            streamer = text_streamer, 
            max_new_tokens = 800,
            use_cache = True,
            # pad_token_id = tokenizer.eos_token_id,
            # eos_token_id = tokenizer.eos_token_id,
        )
        
    else:
        raise ValueError("Invalid model_id")
    
    
    # outputs = model.generate(
    #     input_ids = inputs,
    #     max_new_tokens = 32,
    #     use_cache = True,
    #     # pad_token_id = tokenizer.eos_token_id,
    #     # eos_token_id = tokenizer.eos_token_id,
    # )
    
    # response = tokenizer.batch_decode(outputs)[0]
    # print(response)
    # # # print("Extracting..")
    # # parsed_response = extract_content(response)
    # # pprint.pprint(f"Query: {query}, Time: {time.time() - start_time}")
    # # print(parsed_response)
    
    # # parsed_response_dict = eval(parsed_response)
    
    # # return parsed_response, parsed_response_dict
    
    

In [18]:


for model_id, (judge_model, judge_tokenizer) in judge_models_and_tokenizers.items():
    print(f"Judge: {model_id}")
    for i, response in enumerate(responses):
        print(f"{i:03d}/{len(responses):03d}, {response['Input']}")
        input_text = response["Input"]
        gt_response = response["Reference"]
        pd_response = response["Candidate"]
        judge(model_id, judge_model, judge_tokenizer, input_text, gt_response, pd_response)


Judge: Saxo/Linkbricks-Horizon-AI-Korean-Gemma-2-sft-dpo-27B
000/016, 지금 옆반 에어컨 상태 알려줘


/usr/local/lib/python3.11/site-packages/bitsandbytes/autograd/_functions.py:315: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


{
    "Input": "지금 옆반 에어컨 상태 알려줘",
    "Structural Coherence": ['두 입력은 구조적으로 일치한다.', 1],
    "Structural Correctness": ['Candidate의 json 구조가 올바르다.', 1],
    "Input Semantic Parsing Coherence": ['필요한 모든 정보를 올바르게 추출했다.', 1],
    "Strategy Coherence": ['Objective과 Expected outputs는 올바르게 추출했지만, step의 일부가 누락되었다. 데이터를 계산하지 않고 반환한다.', 0.8],
    "Instruction Set type q Correctness": ['쿼리 3개중 2개가 실행 가능하다. SELECT * from ABC 쿼리에 맞지 않는 테이블이 있다.', 0.67],
    "Instruction Set type q Coherence": ['실행가능한 쿼리 2개중 2개가 의미적으로 일치한다. Candidate의 SELECT IDU from data_t는 Reference의 SELECT IDU_ID from data_t 와 다르다.', 1.0],
    "Instruction Set type o Correctness": ['파이썬 스크립트 1개중 1개가 실행 가능하다. np.addtwo()는 없는 함수다.', 1.0],
    "Instruction Set type o Coherence": ['실행가능한 파이썬 스크립트 1개중 1개가 의미적으로 일치한다. pd.groupby()는 pd.sum()과 의미적으로 다르다.', 1.0],
    "Instruction Set type r Coherence": ['응답 1개중 1개가 의미적으로 일치한다.', 1.0]
}<eos>
001/016, 이번주 우리반 평균 온도 알려줘
{
    "Input": "이번주 우리반 평균 온도 알려줘",
    "Structural Coherence": ['두 입력은

KeyboardInterrupt: 